In [5]:
import GloVepreprocessing
import model

import numpy as np
import logging

import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Lambda 
from keras.layers import Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Reshape, LSTM, Embedding, TimeDistributed
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
import keras.backend as K

In [6]:
preprocessor = GloVepreprocessing.GloVepreprocessor()

In [3]:
vocab_size = preprocessor.weights.shape[0]
emb_size = preprocessor.weights.shape[1]
print(vocab_size)
print(emb_size)

400004
50


In [6]:
sentences = ["Il etait une fois", "Y'en a marre de cette chanson la!"]

embedded_list = preprocessor.GloVe_embed(sentences, preprocessor.weights, isInputSentence=False)
print(embedded_list)
print(np.reshape(embedded_list, (2,35,50)))

[array([[ 0.93332   ,  1.0643    , -0.16746   , ..., -0.22515   ,
        -0.034824  ,  0.29483   ],
       [-0.26339   ,  0.31481   , -0.0488    , ...,  0.34347   ,
         0.68516   ,  1.0541    ],
       [ 0.57657   ,  0.73954   , -1.3782    , ...,  0.34638   ,
         1.2316    ,  1.2587    ],
       ...,
       [-0.18954787, -0.48538032, -0.9108981 , ..., -1.2836249 ,
         0.34865448, -0.6440943 ],
       [-0.18954787, -0.48538032, -0.9108981 , ..., -1.2836249 ,
         0.34865448, -0.6440943 ],
       [-0.18954787, -0.48538032, -0.9108981 , ..., -1.2836249 ,
         0.34865448, -0.6440943 ]], dtype=float32), array([[ 0.58195716, -2.106812  , -0.6334318 , ...,  0.04463232,
         0.6597592 , -0.33220747],
       [ 0.21705   ,  0.46515   , -0.46757   , ..., -0.043782  ,
         0.41013   ,  0.1796    ],
       [ 0.081313  , -0.27803   , -0.15664   , ..., -1.1776    ,
         0.85474   ,  0.52375   ],
       ...,
       [-0.18954787, -0.48538032, -0.9108981 , ..., -1.283

In [7]:
my_batch = preprocessor.batch_generator("train",0, 20)
print(len(my_batch[1]["caption_output"]))

20


In [9]:
#preprocessor = GloVepreprocessing.GloVepreprocessor()

X, Y  = preprocessor.batch_generator("train", 0, 100)


mymodel = model.TrainShowAndTell(preprocessor.MAX_SEQUENCE_LENGTH, preprocessor.VOCAB_SIZE, preprocessor.EMBEDDING_SIZE, 60, preprocessor.weights)

mymodel.compile(loss='categorical_crossentropy', optimizer=Adam(lr = 0.01), metrics=['accuracy'])

a0 = np.zeros((X["image_input"].shape[0], 60))
c0 = np.zeros((X["image_input"].shape[0], 60))

mymodel.fit([X["image_input"],X["caption_input"]
             , a0, c0], Y["caption_output"], batch_size=10, epochs=9, verbose=2)

(None, 60)
(None, 1, 50)
(None, 35, 50)
(None, 35, 50)
Epoch 1/9


ResourceExhaustedError:  OOM when allocating tensor with shape[51200,8192] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[node add_381 (defined at /opt/conda/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_41298]

Function call stack:
keras_scratch_graph


In [ ]:


# Attempt at a simple embedding model
# embeddingmatrix - VOCAB_LENGTH x EMBEDDING_DIMENTION matrix
def GloVe_embed(caption_max_size, embeddingmatrix):
    vocab_size = embeddingmatrix.shape[0]
    emb_size = embeddingmatrix.shape[1]
    
    caption = Input(shape=(caption_max_size,), dtype= "int32", name="caption_input")
    #caption = Lambda(lambda x : K.expand_dims(x, axis=1))(caption)  
    # load GloVe pre-trained word embeddings into an Embedding layer
    # we set trainable = False so as to keep the embeddings fixed
    print(caption.shape)
    embedded_caption = Embedding(vocab_size, emb_size, input_length = caption_max_size, embeddings_initializer = Constant(embeddingmatrix),  mask_zero = False, trainable = False, name = 'emb_text')(caption)
    print(caption.shape)
    print(embedded_caption.shape)

    #return Model(inputs=[caption], outputs=embedded_caption, name='GloVeEmbedding')
    return embedded_caption

In [ ]:
# Call of the embedding model... Fail 
# Also, add the adequate imports

tf.compat.v1.reset_default_graph

sentences = ["Il etait une fois", "Y'en a marre de cette chanson la!"]
mylist = get_sentences_ids(sentences, tokenizer, isInputSentence=True)
mylist = np.array(mylist).reshape(2,35)
print(mylist)
with tf.compat.v1.Session() as test:
    np.random.seed(1)
    caption = tf.compat.v1.placeholder(dtype="int32", shape=(35,))
    print(caption.shape)
    embeddings = GloVe_embed(35, weights)
    test.run(tf.compat.v1.global_variables_initializer())
    out = test.run(embeddings, feed_dict={caption: mylist, K.learning_phase(): 0})
    print(str(out))